# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
!pip install syft

In [0]:
import torch as th

In [0]:
x = th.Tensor([1,2,3,4,5])
x

tensor([1., 2., 3., 4., 5.])

In [0]:
y = x + x
y

tensor([ 2.,  4.,  6.,  8., 10.])

In [0]:
import syft as sy

W0810 08:38:24.873521 139814965557120 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0810 08:38:24.894321 139814965557120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [0]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [0]:
hook

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [0]:
bob._objects

{96367864047: tensor([1, 2, 3, 4, 5])}

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.location == bob

True

In [0]:
x.id_at_location

50844634909

In [0]:
x.id

90558738810

In [0]:
x.id_at_location

96367864047

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:90558738810 -> bob:96367864047]

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# try this project here!

In [0]:
alice = sy.VirtualWorker(hook, id="alice") 
alice

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.Tensor([1, 2, 4, 5, 6])
x

tensor([1., 2., 4., 5., 6.])

In [0]:
x_prt = x.send(bob, alice)
x_prt

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:43184271684 -> bob:54675654236]
	-> [PointerTensor | me:33350998635 -> alice:89189892837]

In [0]:
x_prt.child

MultiPointerTensor>{'bob': [PointerTensor | me:43184271684 -> bob:54675654236], 'alice': [PointerTensor | me:33350998635 -> alice:89189892837]}

In [0]:
x_prt.child.child

{'alice': [PointerTensor | me:33350998635 -> alice:89189892837],
 'bob': [PointerTensor | me:43184271684 -> bob:54675654236]}

In [0]:
x_prt.get()

[tensor([1., 2., 4., 5., 6.]), tensor([1., 2., 4., 5., 6.])]

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:29080183678 -> bob:10726270142]

In [0]:
y

(Wrapper)>[PointerTensor | me:53370714893 -> bob:12171192296]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:44257299423 -> bob:37656852817]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:14595912574 -> bob:34673055538]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:9434128820 -> bob:91496372549]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
# try this project here!

In [0]:
input = th.tensor([[1.0,1.0],[1.0,0.], [0.,1.], [0.,0.]], requires_grad=True).send(bob)
input

(Wrapper)>[PointerTensor | me:37462489706 -> bob:8231117917]

In [0]:
target = th.tensor([[1.0,],[1.0,], [0.,], [0.,]], requires_grad=True).send(bob)
target 

(Wrapper)>[PointerTensor | me:68139498229 -> bob:82133739108]

In [0]:
weigths = th.tensor([[0.,], [0.,]], requires_grad=True).send(bob)
weigths

(Wrapper)>[PointerTensor | me:70386452459 -> bob:95331276334]

In [0]:
pred = input.mm(weigths)
pred

(Wrapper)>[PointerTensor | me:39220792774 -> bob:70652273962]

In [0]:
loss = ((pred - target)**2).sum()
loss

(Wrapper)>[PointerTensor | me:19558252000 -> bob:77094109841]

In [0]:
loss.backward()

(Wrapper)>[PointerTensor | me:44153763586 -> bob:65757942716]

In [0]:
weigths.data.sub_(weigths.grad * 0.01)
weigths.grad *= 0

In [0]:
loss.get().data

tensor(2.)

In [0]:
for i in range(10):
  pred = input.mm(weigths)
  loss = ((pred - target)**2).sum()
  loss.backward()
  weigths.data.sub_(weigths.grad * 0.01)
  weigths.grad *= 0
  print(loss.get().data)

tensor(1.8056)
tensor(1.6323)
tensor(1.4777)
tensor(1.3397)
tensor(1.2165)
tensor(1.1062)
tensor(1.0076)
tensor(0.9193)
tensor(0.8401)
tensor(0.7690)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{27683143208: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x.child.garbage_collect_data

True

In [0]:
bob._objects

{73632963191: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x.child.garbage_collect_data = False

In [0]:
x.child.garbage_collect_data

False

In [0]:
x = "shudipto"

In [0]:
bob._objects

{49486985801: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [0]:
bob._objects

{49486985801: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{49486985801: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{32965463470: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{32965463470: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(8.3618)
tensor(1.0105)
tensor(0.3269)
tensor(0.1907)
tensor(0.1249)
tensor(0.0834)
tensor(0.0561)
tensor(0.0380)
tensor(0.0259)
tensor(0.0178)
tensor(0.0123)
tensor(0.0086)
tensor(0.0061)
tensor(0.0043)
tensor(0.0031)
tensor(0.0022)
tensor(0.0016)
tensor(0.0012)
tensor(0.0009)
tensor(0.0006)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [0]:
train()

tensor(2.8728, requires_grad=True)
tensor(0.8228, requires_grad=True)
tensor(0.5523, requires_grad=True)
tensor(0.5269, requires_grad=True)
tensor(0.2965, requires_grad=True)
tensor(0.3052, requires_grad=True)
tensor(0.1739, requires_grad=True)
tensor(0.1758, requires_grad=True)
tensor(0.1031, requires_grad=True)
tensor(0.1013, requires_grad=True)
tensor(0.0614, requires_grad=True)
tensor(0.0586, requires_grad=True)
tensor(0.0368, requires_grad=True)
tensor(0.0340, requires_grad=True)
tensor(0.0222, requires_grad=True)
tensor(0.0198, requires_grad=True)
tensor(0.0135, requires_grad=True)
tensor(0.0116, requires_grad=True)
tensor(0.0083, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0051, requires_grad=True)
tensor(0.0042, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0009, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
bob._objects

{65685497869: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{53133194659: (Wrapper)>[PointerTensor | alice:53133194659 -> bob:65685497869]}

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[PointerTensor | me:48920685878 -> alice:83765753494]

In [0]:
bob._objects

{12377431492: tensor([ 2,  4,  6,  8, 10]),
 65685497869: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{53133194659: (Wrapper)>[PointerTensor | alice:53133194659 -> bob:65685497869],
 83765753494: (Wrapper)>[PointerTensor | alice:83765753494 -> bob:12377431492]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{79733606430: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{37699437896: (Wrapper)>[PointerTensor | alice:37699437896 -> bob:79733606430]}

In [0]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:37699437896 -> bob:79733606430]

In [0]:
bob._objects

{79733606430: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{63729893464: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{34740069857: (Wrapper)>[PointerTensor | alice:34740069857 -> bob:63729893464]}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{505281950: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x.move(alice)

(Wrapper)>[PointerTensor | me:97593750630 -> alice:71541230054]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{71541230054: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{51867631961: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{71541230054: tensor([1, 2, 3, 4, 5]),
 72750252394: (Wrapper)>[PointerTensor | alice:72750252394 -> bob:51867631961]}

In [0]:
x.remote_get()

(Wrapper)>[PointerTensor | me:82864778277 -> alice:72750252394]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{71541230054: tensor([1, 2, 3, 4, 5]), 72750252394: tensor([1, 2, 3, 4, 5])}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:90642140335 -> bob:82864778277]

In [0]:
bob._objects

{82864778277: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{71541230054: tensor([1, 2, 3, 4, 5])}